In [255]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats
from scipy.special import factorial
import scipy.optimize as opt
import math
# import csv
from matplotlib import rc, rcParams

In [ ]:
with open('DorothyFileC050.txt', mode ='r') as file: # "ElsieWiddowsonFileC051.txt"
    line = file.readline()
    while line[0] == "#":
        print("HEADER" + line)
        line = file.readline()
        # print header
    # Check to make sure first line of data is correct
    data1 = [line.split(sep="\t")]
    print("First line of data in file: " + str(data1))
    data1[0][3] = int(data1[0][3])
    # Check time is at this index
    time = data1[0][3]
    numIntervals = 2000
    intervalTime = 10 #Range in seconds

    # Alex 18/09 NOTE This function will crash for index out of range, and may run indefinitely in the case of data with bad time.
    while time < (numIntervals * intervalTime * 1000): # Data is in ms 
        data1 += [file.readline().split()]
        data1[-1][3] = int(data1[-1][3])
        time = data1[-1][3]

# Bin function graciously borrowed from pythonForJLAB https://github.mit.edu/juniorlab/Python-Intro/blob/master/pythonForJLAB.ipynb

# Take out zero bins. The discussion for why I do this is beyond the scope of this intro, ask the instructors
# It's not important to know what's going on in this function if you don't want to know.
# If you do want to know, you'll have to think about it
# Alex 18/09 For a time interval of 10s, its really unlikely that we will get empty bins but we should do this for robustness.
def delete_zeros(bins,counts,err):
    '''
    Inputs:
    bins = the frequency bin centers
    counts = the frequency data
    err = the error data
    Output:
    new_bins = bin centers, but if the frequency of a bin center is zero the bin is removed
    new_counts = frequency data, but if "                    "
    new_err = error data, but if "                       "
    '''
    zeros = np.where(counts==0) # Find the indices where the frequency data is zero
    mask = np.ones(len(counts),dtype=bool) # create a mask of True values
    mask[zeros[0]] = False # Turn the zero parts of the mask False
    print(counts[mask])
    new_counts = counts[mask] # Recreate the bin data without the False parts
    new_bins = bins[mask] # Recreate the frequency data without the False parts
    new_err = err[mask] # Recrete the "      "
    return new_bins, new_counts, new_err

print(len(data1))
print("First data point at : " + str(data1[0]))
print("Last data point at : " + str(data1[-1]))

ind = 0
bin_counts = []
# Left edge must start at time = 0, right most edge is our recording limit so will be the number of intervals * intervalTime * unit conversion
bin_edges_1 = np.arange(0,numIntervals + 1)

for interval in range(1, numIntervals + 1):
    count = 0
    while data1[ind][3] < interval * 1000 * intervalTime:
        count += 1
        ind += 1

    bin_counts += [count]

plt.rcParams["figure.figsize"] = (15,5)
print(bin_counts)
filter_bin_centers, filter_counts, filter_err = delete_zeros( bins = (np.arange(0,numIntervals) + np.ones(numIntervals)*0.5), counts = np.array(bin_counts), err = np.sqrt(bin_counts))
# plt.bar takes bin centers as its arg, not edges.
plt.bar(filter_bin_centers, filter_counts, width = 1, yerr=filter_err)
plt.xlabel(r'Time ({0}s)'.format(intervalTime), fontsize=20, labelpad=20)
plt.ylabel(r'Count in bin', fontsize=20)
plt.legend()
plt.show()
print(filter_bin_centers)
print(filter_counts)
print(filter_err)
# Update num intervals to num of non zero bins
numIntervals = len(filter_counts)
# lines = [line.strip() for line in lines]
# data = [int(line) for line in lines if line.isdigit()]
# print(data)

In [257]:
# Plotting cumulative mean and demonstrating the reduction in error
cumulativeaverage = np.cumsum(filter_counts) / np.arange(1, numIntervals + 1)
# For Poisson, Var = mean, so this is the std on the mean. Err per bar is stored in filter_err
stds = np.sqrt(cumulativeaverage / np.arange(1, numIntervals + 1))

In [ ]:
plt.errorbar(np.arange(1, numIntervals+1), cumulativeaverage, yerr = stds, lw=0.3)
plt.xlabel(r'# of Measurements ({0}s intervals)'.format(intervalTime), fontsize=20, labelpad=20)
plt.ylabel(r'Cumulative average count per interval', fontsize=20)
plt.show()

In [ ]:
# Also from PythonForJLAB
def poisson(x, lam, a):
    '''
    Inputs:
    a = normalization constant
    x = corresponding number of events, should be an array
    lam = expected value, which is equal to the variance for poisson distribution
    Outputs: probability of x number of events occuring, scaled by constant a
    '''
    return a * lam**x * np.exp(-lam)/factorial(x)

count_range = (filter_counts.min(),filter_counts.max()+1)

binvalues, binedges2, patches = plt.hist(filter_counts, np.arange(count_range[0], count_range[1]))
print("vals: {0}, edges: {1} ".format(len(binvalues), len(binedges2)))
print("bin values: " + str(binvalues))
x = np.linspace(filter_counts.min()-1,filter_counts.max()+1,100)

plt.plot(x, poisson(x, cumulativeaverage[-1], numIntervals))
plt.xlabel(r'Counts per interval (Counts / {0}s)'.format(intervalTime), fontsize=20, labelpad=20)
plt.ylabel(r'Frequency', fontsize=20)
plt.show()
centers = binedges2[0:len(binedges2)-1] + np.ones(len(binedges2)-1)*0.5
err = np.sqrt(binvalues)
# Remove new zero bins from hsit
new_cent, new_val, new_err = delete_zeros(centers, binvalues, err)

In [ ]:
#np.sum((poisson(bincenters, cumulativeaverage[-1], numIntervals * 50 / 29) - binvalues[0]) ** 2 / ((np.sqrt(cumulativeaverage[-1])) ** 2))
print(*new_cent)
print(*new_val)
print(*new_err)
popt, pcov = opt.curve_fit(f=poisson, xdata=new_cent, ydata=new_val, sigma=new_err, p0=[5,200], absolute_sigma=True, maxfev = 100000) # Fit function
uncert = np.sqrt(np.diag(pcov)) # Extract uncertainty from fit
plt.errorbar(new_cent, new_val, yerr = new_err, linewidth=1, ls='none', fmt='o', capsize=3) # Plot actual data
x = np.linspace(start=new_val.min()-5,stop=new_val.max()+5,num=100,endpoint=True) # Define x for smoother plotting of our poisson fit function

plt.plot(x, poisson(x, popt[0], popt[1])) # Plot fit function
plt.xlabel(r'Counts per interval with error(Counts / {0}s)'.format(intervalTime), fontsize=20, labelpad=20)
plt.ylabel(r'Frequency', fontsize=20)
plt.title(r'Poisson fit curve with plotted data included error', fontsize = 25)
plt.show()
print("popt, pcov, uncert: " + str(popt) +", "+ str(pcov) +","+ str(uncert))

In [ ]:
# Time to calculate chi2

def chisq(func,popt,x,y,sig):
    '''
    Inputs:
    func = function to generate expected value
    x = x data
    y = y data
    sig = sigma data
    Outputs:
    chi2 = chi-squared value
    '''
    expected_vals = func(x, *popt) # Again, better off using *popt
    return np.sum((y-expected_vals)**2/sig**2)

chi2 = chisq(poisson,popt,new_cent,new_val,new_err)
print('Calulated Chi2: {0}',format(chi2))
dof = len(new_cent)-2
chi2_probs = 1 - stats.chi2.cdf(chi2,dof)
print('Chi2 probability: {0}'.format(chi2_probs))
